<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:10px 5px'> 
Master Thesis Yannik Haller - Data Preprocessing VADER
</h1>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
1. Load required packages and the data
</h2>
</div>

In [1]:
# Import required baseline packages
import re
import os
import glob
import time
import sys
import pandas as pd
import numpy as np
from pprint import pprint

# Change pandas' setting to print out long strings
pd.options.display.max_colwidth = 200

# Spacy (for lemmatization)
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

In [2]:
# Set the appropriate working directory
os.chdir('D:\\Dropbox\\MA_data')

In [3]:
# Read in the aggregated data
de_tx = pd.read_csv("agg_csv_sparse_de.csv", index_col = 0, dtype = {'so': object, 'la': object, 'tx': object})

In [ ]:
# Take a look at the shape of the data
de_tx.shape

(1934313, 3)

In [ ]:
# Store the article IDs (i.e. index)
de_idx = de_tx.index  # German

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2. Preprocess the text data
</h2>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2.1 Define all required functions to preprocess the data
</h2>
</div>

In [ ]:
# Define a function to prepare/pre-clean the text data for the sentiment analysis using the VADER classifier
def pre_clean_vader(articles):
    # Raise an error if an inappropriate data type is given as an input
    if(not isinstance(articles, list)):
        raise ValueError("Invalid input type. Expected a list.")

    # Keep track of the processing time
    t = time.time()

    # Remove any links starting with http:// or https://
    articles = [re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+').sub('',x) for x in articles]
    # Remove any links starting with www.
    articles = [re.compile('www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+').sub('',x) for x in articles]

    # Replace punctuations which are not followed by a blank with punctuations followed by a blank
    articles = [re.sub(r'[\.]', '. ', x) for x in articles]
    # Separate words in which a lowercase letter is followed by a capital letter, since they usually do not belong together
    articles = [re.sub('(^[a-z]*)+([A-Z])', r'\1 \2', x) for x in articles]
    # Correct manually for those cases where a name like 'McDonalds' was separated to Mc Donalds
    articles = [re.sub('Mc ', 'Mc', x) for x in articles]
    # Replace quotation marks with a blank
    articles = [re.sub('«', ' ', x) for x in articles]
    articles = [re.sub('»', ' ', x) for x in articles]
    # Remove percentage signs
    articles = [re.sub('%', ' ', x) for x in articles]
    # Remove distracting hyphens
    articles = [re.sub("-", " ", x) for x in articles]
    articles = [re.sub("–", " ", x) for x in articles]
    # Remove any blank that precedes a comma
    articles = [re.sub(" ,", ",", x) for x in articles]
    # Remove any blank that precedes a dot
    articles = [re.sub(" \.", ".", x) for x in articles]

    # Replace control characters (e.g. \n or \t) and multiple blanks with a single blank
    articles = [re.sub('\s+', ' ', x) for x in articles]
    # Print out the processing time
    print("Processing time for pre-cleaning: ", str((time.time() - t)/60), "minutes")

    # Return the pre-cleaned text data
    return articles

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2.2 Apply the preprocessing
</h2>
</div>

In [ ]:
# Apply the preprocessing by means of the previously defined function
de_tx = pre_clean_vader(de_tx.tx.values.tolist())

Processing time for pre-cleaning:  14.28923449118932 minutes


In [ ]:
de_tx[0]

' Rückkehrer Stefan Meier überragt beim 7:6 gegen die Flames. Herisau bangt allerdings am Schluss. Lukas PfiffnerIn der vergangenen Saison tat sich der UHC Herisau darin hervor, immer wieder einen Rückstand aufzuholen und Partien zu kehren. In der noch jungen 1. Liga Meisterschaft 2020/21 lebt das Team mindestens in den Heimspielen einem neuen Trend nach: trotz deutlicher Führung noch zu zittern. Am Samstag lagen die überzeugenden Ausserrhoder 2:0 vorne, sie reagierten auf den Ausgleich der Flames mit drei Toren innert dreier Minuten. Sie besassen mit Stefan Meier, der während neun Saisons für Wasa verteidigt hat und im Sommer aus der NLA zu seinem Stammverein zurückgekehrt ist, einen herausragenden Stürmer. Elf Minuten vor der Sirene hiess es 6:3, zum fünften Mal hatte Meier seinen Stock im Spiel. Mit der Sicherheit am Ball ging allerdings auch die Führung verloren. Der komplette Zusammenbruch drohte. Die Flames konnten aber die Gewichte nicht total verschieben und ein eindrücklicher 

In [ ]:
# Compare the fully preprocessed data to the initial text

'Rückkehrer Stefan Meier überragt beim 7:6 gegen die Flames. Herisau bangt allerdings am Schluss.Lukas PfiffnerIn der vergangenen Saison tat sich der UHC Herisau darin hervor, immer wieder einen Rückstand aufzuholen und Partien zu kehren. In der noch jungen 1.-Liga-Meisterschaft 2020/21 lebt das Team mindestens in den Heimspielen einem neuen Trend nach: trotz deutlicher Führung noch zu zittern.Am Samstag lagen die überzeugenden Ausserrhoder 2:0 vorne, sie reagierten auf den Ausgleich der Flames mit drei Toren innert dreier Minuten. Sie besassen mit Stefan Meier, der während neun Saisons für Wasa verteidigt hat und im Sommer aus der NLA zu seinem Stammverein zurückgekehrt ist, einen herausragenden Stürmer. Elf Minuten vor der Sirene hiess es 6:3, zum fünften Mal hatte Meier seinen Stock im Spiel. Mit der Sicherheit am Ball ging allerdings auch die Führung verloren. Der komplette Zusammenbruch drohte. Die Flames konnten aber die Gewichte nicht total verschieben – und ein eindrücklicher Effort von Niklas Hess trug den Gastgebern den Sieg ein (57.).Der Trainer sagte: «Grosses Kino»Schon eine Woche zuvor hatte Meier bei der einzigen Herisauer Niederlage (5:7 gegen Pfannenstiel Egg) drei Treffer erzielt. Er liegt mit sechs Toren und vier Assists nun auf Platz fünf der Skorer in der Gruppe 2. Meier bewegt sich – gemessen an 190\xa0cm Grösse und 82 kg Gewicht – erstaunlich geschmeidig, er kann sich am Ball behaupten und weist einen wuchtigen Schuss auf. «Wenn es so läuft wie aktuell, ist es vorne natürlich schön», meinte der 29-Jährige. Er harmonierte mit seinen Linienpartnern Joel Conzett und Silas Stucki vorzüglich. «Beide sind kreativ.» Trainer Nico Raschle sprach von «grossem Kino», was Meiers Auftritt am Samstag und allgemein seine Einstellung betreffe: Er sei nicht in die 1. Liga gekommen, um seine Karriere einfach ein wenig «ausplämperlen» zu lassen. Dies bestätigt Meier indirekt. «Es war ganz gut, eine neue Position zugeteilt zu bekommen.» Da könne man sich nochmals richtig «reinhängen». Warum spielt Meier in Herisau vorne? «Weil wir uns von ihm das erhoffen, was ihm heute gelungen ist», sagte Raschle. Im resultatmässigen Notstand muss man auch einmal einen Entscheid rückgängig machen: Die Flames hatten innert 90 Sekunden drei Tore geschossen, der Trainer nahm ein Time-out und beorderte Meier für die letzten vier Minuten in die Verteidigung zurück. Ein Problem sei die kurzfristige Umstellung nicht gewesen, meinte dieser. «In der Abwehr machst du vieles über die Routine und das Positionsspiel.» Er habe zudem jahrelang mit Torhüter Dominic Jud im Rücken verteidigt. «Ich kenne seine Anweisungen genau.»Am Sonntagabend CupeinsatzFür Herisau brachte der Samstag im vierten Spiel den dritten Erfolg. Es belegt den zweiten Platz (2,25 Punkte pro Partie) hinter Bassersdorf Nürensdorf (2,5). Seit dieser Saison werden nicht mehr die effektiven Punkte für die Erstellung der Tabelle berücksichtigt, sondern die Quotienten. Bisher gab es in der Gruppe 2 trotz Corona allerdings noch keine Spielabsagen. Am Sonntagabend traten die Ausserrhoder beim 2.-Ligisten Grabs-Werdenberg zum Cupspiel an.Herisau – Flames 7:6 (1:0, 1:2, 5:4)Sportzentrum. – 98 Zuschauer. – Sr. Cereda/Locatelli. Tore: 15. S. Meier (Penalty) 1:0. 22. S. Meier (S. Stucki) 2:0. 25. Mattsson (Bernet) 2:1. 37. Liechti (Jenny) 2:2. 42. Conzett (S. Meier) 3:2. 44. (43:47) Conzett (S. Meier) 4:2. 45. (44:31) Germann (Brandes) 5:2. 49. (48:21) Swoboda 5:3. 49. (48:59) S. Meier (S. Stucki) 6:3. 55. (54:10) B. Jud (Mattsson, Ausschluss Schilling) 6:4. 55. (54:39) Dürr (J. Jud) 6:5. 56. (55:40) Mattsson (Rautio) 6:6. 57. (56:44) Hess 7:6.Herisau: D. Jud; Brunner, Schwarz; Rüegg, Schmid; Stern, L. Stucki; Schweizer; Hess, Schilling, Sandmeier; S. Stucki, Conzett, S. Meier; Germann, Mittelholzer, Wetter; Brandes. Strafen: je 1-mal 2 Minuten.Konzentration bis am Schluss: Herisaus Torhüter Dominic Jud sieht einen Schuss auf sich zukommen. Bild: Lukas Pfiffner'

In [ ]:
# Create a correctly indexed dataframe containing the preprocessed fulltext data in a column and export it as a csv file
pd.DataFrame(de_tx, index = de_idx, columns = ['tx']).to_csv("Preprocessed/Sentiment_Analysis/Fulltext/de_fulltext_senti.csv", index = True, encoding = 'utf-8-sig')
pd.DataFrame(de_tx, index = de_idx, columns = ['tx']).to_csv("Preprocessed/Fulltext/de_fulltext.csv", index = True, encoding = 'utf-8-sig')

In [ ]:
## Create a tsv file which suits the requirements to be fed into the GerVADER algorithm
# Create a correctly indexed dataframe containing the preprocessed fulltext data in a column
de_tx = pd.DataFrame(de_tx, index = de_idx, columns = ['tx'])
# Add a column containing the sentiment (= unknown or un --> Required from GerVADER)
de_tx['senti'] = np.repeat('un', de_tx.shape[0]).tolist()
# Adjust the order of the columns
de_tx = de_tx[['senti','tx']]
# Extract the Dataframe as a tsv file
de_tx.to_csv("Preprocessed/Sentiment_Analysis/Fulltext/de_fulltext_senti.tsv", index = True, encoding = 'utf-8-sig', sep = '\t', header = False)

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
3. Read in and inspect the preprocessed fulltext data
</h2>
</div>

In [ ]:
# Define a function to read in the preprocessed fulltext data
def read_preprocessed_fulltext(language):
    # Raise an error if an inadmissible language is chosen
    allowed_languages = ['de', 'en', 'fr', 'it']
    if language not in allowed_languages:
        raise ValueError("Invalid language. Expected one of: %s" % allowed_languages)
    
    # Set the appropriate working directory
    os.chdir('D:\\Dropbox\\MA_data')

    # Define the name of the file to load
    filename = "Preprocessed/Sentiment_Analysis/Fulltext/"+language+"_fulltext_senti.csv"

    # Read in the data
    tx_ft = pd.read_csv(filename, index_col = 0, dtype = {'tx': object})

    # Get the articles' index together with an enumeration to identify their position in the list of precleaned articles
    idx = tx_ft.index
    idx = pd.DataFrame(idx, columns = [language+'_idx'])

    # Extract the text data as a list of articles
    tx_ft = tx_ft.tx.to_list()

    # Return the preprocessed data
    return tx_ft, idx

In [ ]:
# Read in the preprocessed fulltext data
de_tx_ft, de_idx = read_preprocessed_fulltext('de')

In [ ]:
# Take a look at the first element of the preprocessed fulltext data
de_tx_ft[0]

' Rückkehrer Stefan Meier überragt beim 7:6 gegen die Flames. Herisau bangt allerdings am Schluss. Lukas PfiffnerIn der vergangenen Saison tat sich der UHC Herisau darin hervor, immer wieder einen Rückstand aufzuholen und Partien zu kehren. In der noch jungen 1. Liga Meisterschaft 2020/21 lebt das Team mindestens in den Heimspielen einem neuen Trend nach: trotz deutlicher Führung noch zu zittern. Am Samstag lagen die überzeugenden Ausserrhoder 2:0 vorne, sie reagierten auf den Ausgleich der Flames mit drei Toren innert dreier Minuten. Sie besassen mit Stefan Meier, der während neun Saisons für Wasa verteidigt hat und im Sommer aus der NLA zu seinem Stammverein zurückgekehrt ist, einen herausragenden Stürmer. Elf Minuten vor der Sirene hiess es 6:3, zum fünften Mal hatte Meier seinen Stock im Spiel. Mit der Sicherheit am Ball ging allerdings auch die Führung verloren. Der komplette Zusammenbruch drohte. Die Flames konnten aber die Gewichte nicht total verschieben und ein eindrücklicher 

In [ ]:
# Take a look at the dataframe containing the according index
de_idx

,de_idx
0,16553
1,16554
2,16555
3,16556
4,16557
...,...
1934308,2441178
1934309,2441179
1934310,2441180
1934311,2441181
